In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Завантаження даних
file_path = "./ConcreteStrengthData.csv"

df = pd.read_csv(file_path)

In [4]:
# Визначення ознак (X) та цільової змінної (y)
X = df.drop(columns=["Strength"])
y = df["Strength"]

In [5]:
# Розділення на навчальний та тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Нормалізація даних
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# Перетворення на тензори
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [9]:
# Створення DataLoader для батчового навчання
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [11]:
# Визначення моделі нейронної мережі
class ConcreteStrengthNN(nn.Module):
    def __init__(self):
        super(ConcreteStrengthNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
    
    def forward(self, x):
        return self.model(x)

In [12]:
# Ініціалізація моделі
model = ConcreteStrengthNN()

In [13]:
# Вибір функції втрат та оптимізатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [18]:
# Навчання моделі
epochs = 220
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss / len(train_loader):.4f}")

Epoch [10/220], Loss: 20.8064
Epoch [20/220], Loss: 20.5832
Epoch [30/220], Loss: 20.0785
Epoch [40/220], Loss: 19.3302
Epoch [50/220], Loss: 18.9370
Epoch [60/220], Loss: 18.8091
Epoch [70/220], Loss: 18.8056
Epoch [80/220], Loss: 19.0567
Epoch [90/220], Loss: 18.5043
Epoch [100/220], Loss: 17.7290
Epoch [110/220], Loss: 17.3567
Epoch [120/220], Loss: 17.2038
Epoch [130/220], Loss: 16.7165
Epoch [140/220], Loss: 16.8381
Epoch [150/220], Loss: 16.2271
Epoch [160/220], Loss: 15.8108
Epoch [170/220], Loss: 15.4336
Epoch [180/220], Loss: 15.7514
Epoch [190/220], Loss: 15.4000
Epoch [200/220], Loss: 15.0426
Epoch [210/220], Loss: 15.1105
Epoch [220/220], Loss: 14.5445


In [19]:
# Оцінка моделі
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor).item()

print(f"Test MSE: {test_loss:.4f}")

Test MSE: 33.0099


MSE: 33.0099 є відмінним результатом. Тож за умовами задачі модель досягла високого рівня точності.

MSE тут є оптимальним вибором, оскільки у даних немає значних викидів, а точність прогнозу має пріоритет.